In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import os

def busca_indicadores_acoes(url_statusinvest):
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36 Edg/100.0.1185.36"}
    resposta_ind = requests.get(url_statusinvest, headers = headers)
    sopao_macarronico = resposta_ind.text
    sopa_bonita = BeautifulSoup(sopao_macarronico, 'html.parser')
    lista_indicadores = sopa_bonita.find_all('h3',{'class':re.compile('title m-0.*uppercase')})
    valores_indicadores = sopa_bonita.find_all('strong',{'class':"value d-block lh-4 fs-4 fw-700"})
    
    empresa_com_indicador = []
    for indicadores,valor in zip(lista_indicadores[0:30],valores_indicadores[0:30]):
        nome_indicador = indicadores.text
        n_valor = valor.text
        empresa_com_indicador.append((nome_indicador,n_valor))
    
    df_indicadores = pd.DataFrame(empresa_com_indicador,columns=['indicador', 'valor'])
    return df_indicadores

papel = 'abev3'

def Processar_resposta (resposta,nome):

    df_1 = busca_indicadores_acoes(f'https://statusinvest.com.br/acoes/{resposta}')

    templates = [
        ['P/L','Retorna a quantidade de anos que demora para recuperar o investimento atraves dos dividendos. Ideal seria menos tempo de retorno do que a SELIC (11,75% ano) = 8,51 <> inferior a 9anos. Se superior, preço do ativo esta elevado. Se inferior, preço do ativo esta em conta. Se negativo,  empresa teve prejuizo no periodo'],
        ['P/VP','Quanto o mercado esta disposto a pagar pelo patrimonio liquido da empresa. De 0 a 1 esta sendo negociada abaixo do valor patrimonial (em conta). De 1 a 2 preço justo. Se maior que 2  esta sendo negociada acima do esperado. Se negativo, empresa possui mais passivos do que ativos (contraiu muitas dividas)'],
        ['P/EBIT','Retorna o tempo (em anos) que demora para fazer do lucro operacional o valor de mercado da empresa (considerando que o lucro operacional se mantenha durante este periodo). Valor ideal seria de ate 10 anos, quanto menor melhor. Acima de 10 esta caro. Se inferior a 10 esta sendo negociada abaixo ao valor. Se superior a 10 pode estar sendo negociada superior ao valor. Se negativo, trata se de investimento de alto risco, empresa nao registrou lucro nos ultimos 12 meses'],
        ['P/SR','Tempo (em anos) que demora para fazer em receita liquida o valor de mercado da empresa. Se 1 a 2 o preco esta justo. Se >2 preco do ativo esta elevado. Se <1 preco do ativo esta em conta'],
        ['P/Ativo','Quanto os investidores estão dispostos a pagar por empresa. Se <1= preço do ativo esta em conta, cuidado pois empresas com dificuldades podem apresentar esta faixa. Se 1 a 2 preço ativo está justo. Se >2 preço do ativo esta elevado'],
        ['P/Cap. Giro','O quanto o valor de mercado da empresa representa o capital de giro dela ( qto os acionista estão dispostos a pagar por ela) , quanto menor melhor. Se negativo, empresa possui mais passivos do que ativos circulante, pode apresentar dificuldades para pagar as dívidas de curto prazo. Se = 1 preço do ativo esta em conta (não significa que é um bom negócio)'],
        ['P/Ativo Circ. Liq.','Demosntra a solidez da empresa, se a empresa pagar todas as obrigações do curto e longo prazo , quanto sobraria dos ativo de curto prazo da empresa (caixa, estoque, investimentos). Se Negativo, empresa não conseguiria honrar hoje com todos os seus compromissos, isso nesta data, mas as empresas possuem prazos para honrar com seus compromissos. Se 0 à 1 preço do ativo esta em conta'],
        ['D.Y','Representa o percentual de quanto foi ganho pelos acionistas por cara acao no passado, não é garantia de lucro no futuro (ideal que verifique se possui histórico de pagamentos nos ultimos 5 anos)'],
        ['EV/EBITDA','Tempo que demora para recuperar o valor investido com base no ebtida dos ultimos 12 meses, se o lucro estiver negativo é obrigatório avaliar este indicar, caso o EBITDA também esteja negativo pode indicar graves problemas operacionais. Lucro negativo e EBITDA positivo não tem problemas operacionais mas está pagando valores altos de juros taxas ou impostos. Quanto menor melhor'],
        ['EV/EBIT','Tempo que levara para recuperar o valor investido com base no EBIT dos ultimos 12 meses. Lucro negativo e EBIT positivo não tem problemas operacionais mas está pagando valores altos de juros taxas ou impostos. Quanto menor menor (mais em conta)'],
        ['CAGR Receitas 5 anos','Indicador para avaliar se o faturamento da empresa vem aumentando ou reduzindo ao longo dos anos. Valilda a qualidade do ativo, indica o passado não garante crescimento futuro, importante ser positivo e quanto maior melhor.'],
        ['LPA','Lucro por ação, mostra se a empresa é lucrativa ou não, quanto de lucro consegue fazer por cada ação existente. Ideal comparar entre empresas do mesmo setor. Se for negativo a empresa registrou prejuízo'],
        ['VPA','Quando do patrimônio liquido da empresa o investidor estará se tornando dono por cada ação que ele comprar. Se menor que a metade da cotação, preço do ativo está elevado. Se entre a metade e o valor da cotação, preço está justo. Se maior que a cotação, preço em conta. Se negativo, investimento de alto risco'],
        ['M. Bruta','Mede a lucratividade das vendas de uma emrpesa, possível identificar se a empresa possui vantagem competitiva no mercado em que ela atua. Quanto maior a margem bruta, maior a capacidade de converter suas vendas em lucro. Ideal seria igual ou maior que 40%. Menos que 40%, setores com muita concorrência, podendo afetar a margem de lucro nas dificuldades. Menos de 20%, empresa sem diferencial competitivo ou segmento extremamente competitivo. Se Negativo, empresa registrou prejuízo no período'],
        ['M. EBIT','Mostra o quanto a lucratividade operacional representa na sua receita liquida, possibilita medir o desempenho operacional da empresa, quanto maior melhor. Ideal que seja positivo, melhor margem entre competidores. Se Negativo, empresa registrou prejuízo no período'],
        ['M. Líquida','Quando o lucro liquido representa na receita liquida da empresa. Quanto maior melhor, empresas com vantagem competitiva geram maior margem liquida. Ideal que seja positivo, quanto maior melhor. Se Negativo, empresa registrou prejuízo no período'],
        ['ROIC','Retorno sobre capital investido, quanto uma empresa consegue gerar de lucro com o capital que ela tem investido, quanto maior melhor. Se negativo, emrpesa registrou prejuízo operacional'],
        ['ROE','Retorno sobre patrimônio liquido, rentabilidade da empresa, com base nos recursos próprios, quanto maior melhor, significa que a empresa esta usando seus recursos de maneira bem produtiva. Se negativo a gestão da empresa passa por dificuldades']
    ]

    df_2 = pd.DataFrame(data=templates, columns=['index','descricao'])

    df_final = pd.DataFrame(df_1.join(df_2.set_index('index'),on='indicador',how='inner'))

    print(df_final)
    
    df_final.to_csv("teste.csv")
    
def start():
    #apresentar o chatbot
    print('Olá, somos a Invest_facil')
    #pedir o nome
    nome = input('Qual o seu nome: ')
    #looping
    while True:
        #oferecer opções
        resposta = input(f'{nome}, por favor digite o código da ação que deseja consultar{os.linesep}')
        #processar a resposta enviada
        Processar_resposta(resposta,nome)
        
if __name__ == '__main__':
    start()

Olá, somos a Invest_facil
Qual o seu nome: felipe
felipe, por favor digite o código da ação que deseja consultar
sanb11
               indicador   valor  \
0                    D.Y   7,64%   
1                    P/L    9,22   
3                   P/VP    0,95   
4              EV/EBITDA       -   
5                EV/EBIT   11,20   
7                 P/EBIT   11,20   
8                    VPA   29,88   
9                P/Ativo    0,10   
10                   LPA    3,09   
11                  P/SR    1,13   
12           P/Cap. Giro       -   
13    P/Ativo Circ. Liq.   -0,11   
20              M. Bruta  17,98%   
22               M. EBIT  10,05%   
23            M. Líquida  12,22%   
24                   ROE  10,35%   
26                  ROIC      -%   
28  CAGR Receitas 5 anos  10,04%   

                                            descricao  
0   Representa o percentual de quanto foi ganho pe...  
1   Retorna a quantidade de anos que demora para r...  
3   Quanto o mercado esta d

SyntaxError: invalid syntax (Temp/ipykernel_20424/1942745698.py, line 1)